In [1]:
import pandas as pd
import numpy as np
from scipy import stats


In [2]:
# Load each metric file
mae = pd.read_csv("1_step_mae.csv")
mse = pd.read_csv("1_step_mse.csv")
mape = pd.read_csv("1_step_mape.csv")
rmse = pd.read_csv("1_step_rmse.csv")

# View structure
print(mae.head())


                         Column1 Column2 Column3                Column4  \
0  MAE Results One step Forecast     NaN     NaN                    NaN   
1                            NaN     NaN     NaN                    NaN   
2                           YEAR   ARIMA    VECM  Exponential Smoothing   
3                           2009     3.5    2.87                  12.63   
4                           2010    3.28    5.77                   3.38   

   Unnamed: 4  Unnamed: 5  Unnamed: 6  Unnamed: 7 Unnamed: 8  
0         NaN         NaN         NaN         NaN        NaN  
1         NaN         NaN         NaN         NaN        NaN  
2         NaN         NaN         NaN         NaN        NaN  
3         NaN         NaN         NaN         NaN        NaN  
4         NaN         NaN         NaN         NaN        NaN  


In [3]:
import pandas as pd

# Load each metric file
mae = pd.read_csv("1_step_mae.csv" , skiprows=[0, 1, 2])
mse = pd.read_csv("1_step_mse.csv", skiprows=1)
mape = pd.read_csv("1_step_mape.csv", skiprows=1)
rmse = pd.read_csv("1_step_rmse.csv", skiprows=1)

# Function to clean dataframe
def clean_dataframe(df):
    # Drop columns that are entirely NaN
    df = df.dropna(axis=1, how='all')
    
    # Drop rows that are entirely NaN
    df = df.dropna(axis=0, how='all')
    
    # Drop columns with names like 'Unnamed: X'
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    
    # Reset index
    df = df.reset_index(drop=True)
    
    return df

# Clean all dataframes
mae_clean = clean_dataframe(mae)
mse_clean = clean_dataframe(mse)
mape_clean = clean_dataframe(mape)
rmse_clean = clean_dataframe(rmse)

# View cleaned structure
print("MAE Results:")
print(mae_clean)
print("\nMSE Results:")
print(mse_clean)
print("\nMAPE Results:")
print(mape_clean)
print("\nRMSE Results:")
print(rmse_clean)

# Optionally save cleaned versions
mae_clean.to_csv("1_step_mae_clean.csv", index=False)
mse_clean.to_csv("1_step_mse_clean.csv", index=False)
mape_clean.to_csv("1_step_mape_clean.csv", index=False)
rmse_clean.to_csv("1_step_rmse_clean.csv", index=False)

MAE Results:
    YEAR  ARIMA   VECM  Exponential Smoothing
0   2009   3.50   2.87                  12.63
1   2010   3.28   5.77                   3.38
2   2011   2.48   0.30                   0.44
3   2012   1.89   1.20                   0.93
4   2013   0.11   1.14                   0.73
5   2014   1.18   1.33                   1.93
6   2015   0.08   0.58                   0.54
7   2016   0.27   0.33                   0.01
8   2017   1.11   1.06                   0.90
9   2018  13.95  13.63                  13.66
10  2019  23.27  32.16                  25.77
11  2020   5.31  15.33                   5.37
12  2021   9.09   9.88                   4.65
13  2022   0.31   3.11                   2.90
14  2023   2.12   1.99                   1.03

MSE Results:
    YEAR   ARIMA     VECM  Exponential Smoothing
0   2009   12.23     8.22                  12.63
1   2010   10.77    33.24                  11.40
2   2011    6.14     0.09                   0.19
3   2012    3.56     1.45                

In [4]:
def bootstrap_dm_test(e1, e2, n_bootstrap=10000, loss='mae', seed=42):
    """Bootstrap Diebold-Mariano test for small samples."""
    np.random.seed(seed)
    e1, e2 = np.array(e1, dtype=float), np.array(e2, dtype=float)
    
    if loss == 'mae':
        d = np.abs(e1) - np.abs(e2)
    elif loss == 'mse':
        d = e1**2 - e2**2
    else:
        raise ValueError("loss must be 'mse' or 'mae'")
    
    d_mean_obs = np.mean(d)
    n = len(d)
    
    bootstrap_stats = []
    for _ in range(n_bootstrap):
        idx = np.random.choice(n, size=n, replace=True)
        d_boot = d[idx]
        bootstrap_stats.append(np.mean(d_boot))
    
    bootstrap_stats = np.array(bootstrap_stats)
    p_value = np.mean(np.abs(bootstrap_stats - np.mean(bootstrap_stats)) >= 
                      np.abs(d_mean_obs - np.mean(bootstrap_stats)))
    
    ci_lower = np.percentile(bootstrap_stats, 2.5)
    ci_upper = np.percentile(bootstrap_stats, 97.5)
    
    return d_mean_obs, p_value, (ci_lower, ci_upper)


In [5]:
def wilcoxon_test(e1, e2, loss='mae'):
    """Wilcoxon signed-rank test for paired samples."""
    e1, e2 = np.array(e1, dtype=float), np.array(e2, dtype=float)
    
    if loss == 'mae':
        loss1 = np.abs(e1)
        loss2 = np.abs(e2)
    elif loss == 'mse':
        loss1 = e1**2
        loss2 = e2**2
    else:
        raise ValueError("loss must be 'mse' or 'mae'")
    
    try:
        statistic, p_value = stats.wilcoxon(loss1, loss2, 
                                           alternative='two-sided',
                                           zero_method='wilcox')
    except ValueError:
        return 0, 1.0, 0.0
    
    n = len(loss1)
    diff = loss1 - loss2
    abs_diff = np.abs(diff[diff != 0])
    
    if len(abs_diff) > 0:
        ranks = stats.rankdata(abs_diff)
        sum_ranks = np.sum(ranks[diff[diff != 0] > 0])
        n_nonzero = len(abs_diff)
        effect_size = 1 - (4 * sum_ranks) / (n_nonzero * (n_nonzero + 1))
    else:
        effect_size = 0.0
    
    return statistic, p_value, effect_size


In [6]:
def sign_test(e1, e2, loss='mae'):
    """Sign test for paired samples."""
    e1, e2 = np.array(e1, dtype=float), np.array(e2, dtype=float)
    
    if loss == 'mae':
        loss1 = np.abs(e1)
        loss2 = np.abs(e2)
    elif loss == 'mse':
        loss1 = e1**2
        loss2 = e2**2
    else:
        raise ValueError("loss must be 'mse' or 'mae'")
    
    diff = loss1 - loss2
    n_wins_model1 = np.sum(diff < 0)
    n_wins_model2 = np.sum(diff > 0)
    n_ties = np.sum(diff == 0)
    
    n_total = n_wins_model1 + n_wins_model2
    
    if n_total == 0:
        return n_wins_model1, n_wins_model2, n_ties, 1.0, 0.5
    
    p_value = 2 * min(
        stats.binom.cdf(n_wins_model1, n_total, 0.5),
        1 - stats.binom.cdf(n_wins_model1 - 1, n_total, 0.5)
    )
    
    win_percentage = n_wins_model1 / n_total if n_total > 0 else 0.5
    
    return n_wins_model1, n_wins_model2, n_ties, p_value, win_percentage

In [7]:
def run_all_robustness_tests(e1, e2, model1_name="Model 1", model2_name="Model 2", 
                             loss='mae', n_bootstrap=10000):
    """Run all three robustness tests."""
    print(f"\n{'='*70}")
    print(f"ROBUSTNESS TESTS: {model1_name} vs {model2_name}")
    print(f"Loss Function: {loss.upper()}, Sample Size: {len(e1)}")
    print(f"{'='*70}\n")
    
    results = {}
    
    # Bootstrap DM Test
    print("1. BOOTSTRAP DIEBOLD-MARIANO TEST")
    print("-" * 50)
    boot_stat, boot_pval, boot_ci = bootstrap_dm_test(e1, e2, n_bootstrap, loss)
    results['bootstrap'] = {'statistic': boot_stat, 'p_value': boot_pval, 'ci_95': boot_ci}
    
    better = model1_name if boot_stat < 0 else model2_name
    print(f"Test Statistic (Mean Loss Diff): {boot_stat:.6f}")
    print(f"P-value: {boot_pval:.4f}")
    print(f"95% CI: [{boot_ci[0]:.6f}, {boot_ci[1]:.6f}]")
    print(f"Better Model: {better}")
    print(f"Significance: {interpret_significance(boot_pval)}\n")
    
    # Wilcoxon Test
    print("2. WILCOXON SIGNED-RANK TEST")
    print("-" * 50)
    wilc_stat, wilc_pval, effect = wilcoxon_test(e1, e2, loss)
    results['wilcoxon'] = {'statistic': wilc_stat, 'p_value': wilc_pval, 'effect_size': effect}
    
    print(f"Test Statistic: {wilc_stat:.2f}")
    print(f"P-value: {wilc_pval:.4f}")
    print(f"Effect Size (rank-biserial): {effect:.4f}")
    print(f"Significance: {interpret_significance(wilc_pval)}\n")

    # Sign Test
    print("3. SIGN TEST")
    print("-" * 50)
    n_wins1, n_wins2, n_ties, sign_pval, win_pct = sign_test(e1, e2, loss)
    results['sign'] = {
        'wins_model1': n_wins1, 'wins_model2': n_wins2, 'ties': n_ties,
        'p_value': sign_pval, 'win_percentage': win_pct
    }
    
    print(f"{model1_name} wins: {n_wins1} times")
    print(f"{model2_name} wins: {n_wins2} times")
    print(f"Ties: {n_ties} times")
    print(f"Win Percentage ({model1_name}): {win_pct*100:.1f}%")
    print(f"P-value: {sign_pval:.4f}")
    print(f"Significance: {interpret_significance(sign_pval)}\n")
    
    # Summary
    print("="*70)
    print("SUMMARY OF ALL TESTS")
    print("="*70)
    print(f"Bootstrap DM:  p = {boot_pval:.4f}  {get_stars(boot_pval)}")
    print(f"Wilcoxon Test: p = {wilc_pval:.4f}  {get_stars(wilc_pval)}")
    print(f"Sign Test:     p = {sign_pval:.4f}  {get_stars(sign_pval)}")
    print()

    # Consensus
    sig_count = sum([boot_pval < 0.05, wilc_pval < 0.05, sign_pval < 0.05])
    marg_count = sum([boot_pval < 0.10, wilc_pval < 0.10, sign_pval < 0.10])
    
    if sig_count >= 2:
        print(f"✓✓ STRONG EVIDENCE: {better} is significantly better")
    elif marg_count >= 2:
        print(f"✓ MODERATE EVIDENCE: {better} is marginally better")
    else:
        print("○ WEAK EVIDENCE: No clear winner, models are comparable")
    
    print("="*70 + "\n")
    
    return results
def interpret_significance(p_value):
    """Helper function to interpret p-value"""
    if p_value < 0.01:
        return "Highly Significant (p < 0.01) ***"
    elif p_value < 0.05:
        return "Significant (p < 0.05) **"
    elif p_value < 0.10:
        return "Marginally Significant (p < 0.10) *"
    else:
        return "Not Significant"


def get_stars(p_value):
    """Helper function to get significance stars"""
    if p_value < 0.01:
        return "***"
    elif p_value < 0.05:
        return "**"
    elif p_value < 0.10:
        return "*"
    else:
        return ""

In [9]:
def analyze_by_economic_period(data_clean, metric_name='MAE', loss='mae'):
    
    # Define economic periods for Zimbabwe (2009-2023)
    periods = {
        'Stable': {
            'years': [2009, 2010, 2011, 2012, 2013],
            'description': 'Post-Dollarization Stability (2009-2013)',
            'context': 'Multi-currency system, GDP recovery, inflation control'
        },
        'Transition': {
            'years': [2014, 2015, 2016, 2017, 2018],
            'description': 'Gradual Deterioration (2014-2018)',
            'context': 'Liquidity crisis, bond notes, economic decline'
        },
        'Unstable': {
            'years': [2019, 2020, 2021, 2022, 2023],
            'description': 'High Instability (2019-2023)',
            'context': 'Currency collapse, hyperinflation, COVID-19, continued volatility'
        }
    }
    print("\n" + "="*80)
    print(f"PERIOD-SPECIFIC ANALYSIS: {metric_name}")
    print("="*80)
    print("\nEconomic Context for Zimbabwe (2009-2023):")
    print("-" * 80)
    for period_name, period_info in periods.items():
        print(f"\n{period_name.upper()} PERIOD: {period_info['description']}")
        print(f"Years: {period_info['years']}")
        print(f"Context: {period_info['context']}")
    print("\n" + "="*80)
    
    # Store results for summary table
    summary_results = {}
    
    for period_name, period_info in periods.items():
        period_years = period_info['years']
        period_desc = period_info['description']
        
        # Filter data for this period
        period_data = data_clean[data_clean['YEAR'].isin(period_years)].copy()
        
        if len(period_data) == 0:
            print(f"\n⚠️  No data available for {period_name} period")
            continue
        
        print(f"\n{'#'*80}")
        print(f"# {period_name.upper()} PERIOD: {period_desc}")
        print(f"# Sample Size: {len(period_data)} observations")
        print(f"# Years: {sorted(period_data['YEAR'].tolist())}")
        print(f"{'#'*80}\n")

        # Extract model errors for this period
        arima = period_data['ARIMA'].values
        vecm = period_data['VECM'].values
        exp_smooth = period_data['Exponential Smoothing'].values
        
        # Calculate average errors for quick comparison
        avg_arima = np.mean(np.abs(arima)) if loss == 'mae' else np.mean(arima**2)
        avg_vecm = np.mean(np.abs(vecm)) if loss == 'mae' else np.mean(vecm**2)
        avg_exp = np.mean(np.abs(exp_smooth)) if loss == 'mae' else np.mean(exp_smooth**2)
        
        print(f"Average {metric_name}:")
        print(f"  ARIMA: {avg_arima:.4f}")
        print(f"  VECM: {avg_vecm:.4f}")
        print(f"  Exponential Smoothing: {avg_exp:.4f}")
        print()
        
        # Store for summary
        summary_results[period_name] = {
            'n': len(period_data),
            'avg_mae': {'ARIMA': avg_arima, 'VECM': avg_vecm, 'Exp_Smooth': avg_exp}
        }

        # Run pairwise comparisons
        print(f"\n{'─'*80}")
        print("COMPARISON 1: ARIMA vs VECM")
        print(f"{'─'*80}")
        results_av = run_all_robustness_tests(
            arima, vecm,
            model1_name="ARIMA",
            model2_name="VECM",
            loss=loss,
            n_bootstrap=10000
        )
        summary_results[period_name]['ARIMA_vs_VECM'] = results_av
        
        print(f"\n{'─'*80}")
        print("COMPARISON 2: ARIMA vs Exponential Smoothing")
        print(f"{'─'*80}")
        results_ae = run_all_robustness_tests(
            arima, exp_smooth,
            model1_name="ARIMA",
            model2_name="Exponential Smoothing",
            loss=loss,
            n_bootstrap=10000
        )
        summary_results[period_name]['ARIMA_vs_ExpSmooth'] = results_ae


        print(f"\n{'─'*80}")
        print("COMPARISON 3: VECM vs Exponential Smoothing")
        print(f"{'─'*80}")
        results_ve = run_all_robustness_tests(
            vecm, exp_smooth,
            model1_name="VECM",
            model2_name="Exponential Smoothing",
            loss=loss,
            n_bootstrap=10000
        )
        summary_results[period_name]['VECM_vs_ExpSmooth'] = results_ve
    
    # Print comparative summary
    print_period_comparison_summary(summary_results, metric_name)
    
    return summary_results

In [13]:
def print_period_comparison_summary(results, metric_name):
    """Print a summary table comparing model performance across periods."""
    
    print("\n" + "="*80)
    print(f"COMPARATIVE SUMMARY: MODEL RANKINGS ACROSS ECONOMIC PERIODS")
    print("="*80)
    
    print(f"\n{metric_name} Rankings by Period:")
    print("-" * 80)
    
    # Create ranking for each period
    for period in ['Stable', 'Transition', 'Unstable']:
        if period not in results:
            continue
            
        avg_mae = results[period]['avg_mae']
        n = results[period]['n']
        
        # Sort models by average MAE
        sorted_models = sorted(avg_mae.items(), key=lambda x: x[1])
        
        print(f"\n{period.upper()} Period (n={n}):")
        for rank, (model, mae_val) in enumerate(sorted_models, 1):
            model_display = model.replace('_', ' ')
            print(f"  {rank}. {model_display:25s} {metric_name}={mae_val:.4f}")
   


In [14]:
if __name__ == "__main__":
    
    # Load your cleaned data
    print("Loading data...")
    mae_clean = pd.read_csv("1_step_mae_clean.csv")
    
    print(f"Total observations: {len(mae_clean)}")
    print(f"Year range: {mae_clean['YEAR'].min()} - {mae_clean['YEAR'].max()}")
    
    # Run period-specific analysis
    results = analyze_by_economic_period(
        mae_clean,
        metric_name='MAE',
        loss='mae'
    )
    
    

Loading data...
Total observations: 15
Year range: 2009 - 2023

PERIOD-SPECIFIC ANALYSIS: MAE

Economic Context for Zimbabwe (2009-2023):
--------------------------------------------------------------------------------

STABLE PERIOD: Post-Dollarization Stability (2009-2013)
Years: [2009, 2010, 2011, 2012, 2013]
Context: Multi-currency system, GDP recovery, inflation control

TRANSITION PERIOD: Gradual Deterioration (2014-2018)
Years: [2014, 2015, 2016, 2017, 2018]
Context: Liquidity crisis, bond notes, economic decline

UNSTABLE PERIOD: High Instability (2019-2023)
Years: [2019, 2020, 2021, 2022, 2023]
Context: Currency collapse, hyperinflation, COVID-19, continued volatility


################################################################################
# STABLE PERIOD: Post-Dollarization Stability (2009-2013)
# Sample Size: 5 observations
# Years: [2009, 2010, 2011, 2012, 2013]
################################################################################

Average MAE:
  ARIMA: